In [3]:

#Authentication code to read data from google drive.Run this is you are using google colab and want to acess your google drive files

import os
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/drive')
# os.chdir('drive/My Drive/')

E: Package 'python-software-properties' has no installation candidate
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [59]:
import numpy as np 
import pandas as pd 
import os

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Dense, Embedding, LSTM, Input, Dropout
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re


In [15]:
#Reading the csv files

data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/friends.csv')
pd.set_option('display.max_colwidth',-1)
data.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


,speaker,utterance,emotion,annotation
0,Phoebe,"Oh my God, hes lost it. Hes totally lost it.",non-neutral,2120
1,Monica,What?,surprise,1000130
2,Ross,"Or! Or, we could go to the bank, close our accounts and cut them off at the source.",neutral,3000200
3,Chandler,Youre a genius!,joy,500000
4,Joey,"Aww, man, now we wont be bank buddies!",sadness,40100


In [16]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
#Removing all the stopwords from the text


scene2 = []
from nltk.corpus import stopwords
stopwords_set = list(stopwords.words("english"))
for i in data['utterance']:
  replaced = re.sub(r"{.*?}", '', i)
  #reduced = filter(lambda w: w not in stopwords, re.split(r'\W+', replaced.lower()))
  #reduced=list(reduced)
  #words_without_stopwords = [word for word in replaced if not word in stopwords_set]
  #scene2.append(re.sub(r"{.*?}", '', i))
  #print(replaced)
  scene2.append(replaced)
  
  #print(re.sub(r"{.*?}", '', i))
#print(scene2[0])

scene3 = []  
import re
for j in scene2:
  query = j
  stopwords = stopwords_set

  resultwords  = [word for word in re.split("\W+",query) if word.lower() not in stopwords]
  result = ' '.join(resultwords)
  #print(result)
  scene3.append(result)
print(scene3)

['Oh God lost totally lost ', '', 'could go bank close accounts cut source ', 'genius ', 'Aww man bank buddies ', 'two reasons ', 'Hey ', 'Hey ', 'Ohh guys remember cute client told bit ', '', 'touchy ', '', 'know ', 'sorry moment touch wanna throw old oath take new dirty one ', 'Well next time massaging try distract ', 'Yeah Yeah Yeah Like like something exciting wanna get', 'Thank Joey ', 'thank ', 'Hey Estelle listen', 'Well Well Well Joey Tribbiani came back huh', 'talkin never left always agent ', 'Really ', 'Yeah ', 'Oh well harm foul ', 'Okay guys free tonight ', 'Yeah ', 'Tonight say going nighttime ', 'Yes Yes help ', 'Yeah looking around ', 'Oh oh fellow scholars ', 'exactly looking hmm ', 'Perhaps perhaps Dr Chester Stock musings Smiledon Californicus ', 'Uhh ', 'Ah Ah Get Uh meeting someone brush Marion views evolution ', 'Uh actually find Marion views far progressionist ', 'find Marion views far progressionist ', 'sorry ', 'professor uh Ross Geller ', 'Ross Geller know nam

In [19]:
#Toekenizing the words

num_words = 500
tokenizer = Tokenizer(num_words=num_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')
print(tokenizer)
X=tokenizer.fit_on_texts(scene3)
print(X)
X = tokenizer.texts_to_sequences(scene3)


word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

max_length_of_text = 100
X = pad_sequences(X, maxlen=max_length_of_text)

None
Found 6226 unique tokens.


In [20]:
X[10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [21]:
y=data['emotion']

In [22]:
!pip install category_encoders

     |████████████████████████████████| 81kB 4.2MB/s 


In [24]:
#Onehot encoding all the 7 labels

import category_encoders as ce
le =  ce.OneHotEncoder(return_df=False, handle_unknown="ignore")
#enc= OneHotEncoder(handle_unknown='ignore')
#enc.fit(y)
img_label=np.array(y)
enc=le.fit_transform(img_label)
#image_labels_enc=img_label.reshape(-1,1)
"""enc = OneHotEncoder()
img_fit=enc.fit(image_labels_enc)
img_encode=enc.transform(y).toarray()"""

'enc = OneHotEncoder()\nimg_fit=enc.fit(image_labels_enc)\nimg_encode=enc.transform(y).toarray()'

In [25]:
enc[4]


array([0, 0, 0, 0, 1, 0, 0, 0])

In [26]:
X_train, Xvalid, y_train, yvalid = train_test_split(X,enc, test_size = 0.2, random_state = 42)
print(X_train.shape,y_train.shape)
#print(X_test.shape,y_test.shape)

(11602, 100) (11602, 8)


In [66]:
#LSTM model

embed_dim = 1024
lstm_out = 128
batch_size = 64

inputs = Input((max_length_of_text, ))
x = Embedding(num_words, 50)(inputs)
x = LSTM(64, dropout=0.6, recurrent_dropout=0.6)(x)
x = Dense(32,activation='relu')(x)
x = Dropout(0.6)(x)
x = Dense(8,activation='softmax')(x)
model = Model(inputs, x)
print(model.summary())

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 100, 50)           25000     
_________________________________________________________________
lstm_8 (LSTM)                (None, 64)                29440     
_________________________________________________________________
dense_11 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 8)                 264       
Total params: 56,784
Trainable params: 56,784
Non-trainable params: 0
_______________________________________________________

In [67]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['acc'])

In [68]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
early_stops = EarlyStopping(patience=3, monitor='val_acc',min_delta=0.001)
checkpointer = ModelCheckpoint(filepath='weights4.best.eda.hdf5', verbose=1, save_best_only=True)

In [69]:
model.fit(X_train, y_train,validation_data=(Xvalid, yvalid) ,batch_size = 64, epochs = 50,callbacks=[checkpointer], verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 11602 samples, validate on 2901 samples
Epoch 1/50
11602/11602 [==============================] - 58s 5ms/step - loss: 1.7830 - acc: 0.4015 - val_loss: 1.6023 - val_acc: 0.4526

Epoch 00001: val_loss improved from inf to 1.60234, saving model to weights4.best.eda.hdf5
Epoch 2/50
11602/11602 [==============================] - 57s 5ms/step - loss: 1.6568 - acc: 0.4387 - val_loss: 1.5769 - val_acc: 0.4533

Epoch 00002: val_loss improved from 1.60234 to 1.57693, saving model to weights4.best.eda.hdf5
Epoch 3/50
11602/11602 [==============================] - 57s 5ms/step - loss: 1.6063 - acc: 0.4505 - val_loss: 1.5615 - val_acc: 0.4605

Epoch 00003: val_loss improved from 1.57693 to 1.56146, saving model to weights4.best.eda.hdf5
Epoch 4/50
11602/11602 [==============================] - 57s 5ms/step - loss: 1.5730 - acc: 0.4643 - val_loss: 1.5421 - val_acc: 0.4685

Epoch 00004: val_loss improved from 1.56146 to 1.54210, saving model to weights4.best.eda.hdf5
Epoch 5/50
11602/11602 

In [70]:
model.save('nnfl4.eda.hdf5')

In [71]:
score,acc = model.evaluate(Xvalid, yvalid, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

2901/2901 [==============================] - 1s 458us/step
Score: 1.50
Validation Accuracy: 0.48


In [72]:
from sklearn.metrics import f1_score
train_pred = model.predict(X_train)
train_pred = np.argmax(train_pred, axis=1)
print(train_pred)
#f1_score(y_train, train_pred, average='samples')

[2 2 2 ... 2 2 2]


In [73]:
#Loading the test data

data2 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/en_data.csv', encoding= 'unicode_escape')
data2.head()

,id,i_dialog,i_utterance,speaker,utterance
0,0,0,0,Phoebe,"Alright, whadyou do with him?"
1,1,0,1,Monica,Oh! You're awake!
2,2,0,2,Joey,Then you gotta come clean with Ma! This is not right!
3,3,0,3,Mr. Tribbiani,"Yeah, but this is"
4,4,0,4,Joey,I don't wanna hear it! Now go to my room!


In [74]:

#Preprocessing the test data

from keras.models import load_model
import pandas as pd
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

data = data2
scene = data['utterance']

scene2 = []
for i in scene:
  replaced = re.sub(r"{.*?}", '', i)
  scene2.append(replaced)

stopwords_list = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

scene3 = []
for j in scene2:
  query = j
  resultwords  = [word for word in re.split("\W+",query) if word.lower() not in stopwords_list]
  result = ' '.join(resultwords)
  scene3.append(result)

num_words = 500
tokenizer = Tokenizer(num_words=num_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True,split=' ')
tokenizer.fit_on_texts(scene3)
X = tokenizer.texts_to_sequences(scene3)

word_index = tokenizer.word_index

max_length_of_text = 100
X = pad_sequences(X, maxlen=max_length_of_text)

#Loading the model.Write your own model name here
model = load_model('nnfl4.eda.hdf5')
results = model.predict(X)

results = np.argmax(results, axis=1)

predictions = []

conv = {0: "neutral",
	   1: "non-neutral",
	   2: "joy",
	   3: "surprise",
     4: "anger",
     5: "sadness",
     6: "disgust",
     7: "fear"}

for i in results:
	predictions.append(conv[i])
#Converting the solution into a CSV file
new_df = pd.DataFrame({"Id": [i for i in range(len(predictions))], "Predicted": predictions})
new_df.to_csv("sample.csv", index=False)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
